<a href="https://colab.research.google.com/github/yeansam/APTOS_Messidor-2/blob/master/Data_preprocessing_(FYP_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Define packages

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from tqdm import tqdm_notebook
import time

import warnings
warnings.filterwarnings("ignore")
IMG_SIZE = 512

## Load dataset from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/aptos2019-blindness-detection.zip'

Streaming output truncated to the last 5000 lines.
  inflating: test_images/51323e9d2070.png  
  inflating: test_images/515675001b9e.png  
  inflating: test_images/515877da2def.png  
  inflating: test_images/5168ddccea88.png  
  inflating: test_images/5172c056a687.png  
  inflating: test_images/5177461f5339.png  
  inflating: test_images/51910bcb980d.png  
  inflating: test_images/51a3e9b6a1c0.png  
  inflating: test_images/51bbd82feec8.png  
  inflating: test_images/51de5fb8eccd.png  
  inflating: test_images/51e88694c81f.png  
  inflating: test_images/51f0bf8f1544.png  
  inflating: test_images/51f57fb32840.png  
  inflating: test_images/51fa42dc1415.png  
  inflating: test_images/5276ab33b7b0.png  
  inflating: test_images/52a353760f72.png  
  inflating: test_images/531937254e73.png  
  inflating: test_images/5325b1c22c61.png  
  inflating: test_images/53959e0a750b.png  
  inflating: test_images/53be22c72713.png  
  inflating: test_images/5400549bfe76.png  
  inflating: test_images/

## Load csv into data frames

In [ ]:
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

In [ ]:
print(df_train)
print(df_test)

           id_code  diagnosis
0     000c1434d8d7          2
1     001639a390f0          4
2     0024cdab0c1e          1
3     002c21358ce6          0
4     005b95c28852          0
...            ...        ...
3657  ffa47f6a7bf4          2
3658  ffc04fed30e6          0
3659  ffcf7b45f213          2
3660  ffd97f8cd5aa          0
3661  ffec9a18a3ce          2

[3662 rows x 2 columns]
           id_code
0     0005cfc8afb6
1     003f0afdcd15
2     006efc72b638
3     00836aaacf06
4     009245722fa4
...            ...
1923  ff2fd94448de
1924  ff4c945d9b17
1925  ff64897ac0d8
1926  ffa73465b705
1927  ffdc2152d455

[1928 rows x 1 columns]


## Find dark images

In [ ]:
# find dark images
def find_black_images(file_path, df):
    """
    Creates a column of images that are not black (np.mean(img) != 0)
    INPUT
        file_path: file_path to the images to be analyzed.
        df: Pandas DataFrame that includes all labeled image names.
        column: column in DataFrame query is evaluated against.
    OUTPUT
        Column indicating if the photo is pitch black or not.
    """

    lst_imgs = [l for l in df['id_code']]
    return [1 if np.mean(np.array(Image.open(file_path + img))) == 0 else 0 for img in lst_imgs]


if __name__ == '__main__':
    start_time = time.time()
    trainLabels = pd.read_csv('/content/train.csv')

    trainLabels['id_code'] = [i + '.png' for i in trainLabels['id_code']]
    trainLabels['black'] = np.nan

    trainLabels['black'] = find_black_images('/content/train_images/', trainLabels)
    trainLabels = trainLabels.loc[trainLabels['black'] == 0]
    trainLabels.to_csv('trainLabels_black.csv', index=False, header=True)

    print("Completed")
    print("--- %s seconds ---" % (time.time() - start_time))

Completed
--- 496.17611598968506 seconds ---


## Crop images

In [ ]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

## Extract green channel (CLAHE) (Not using)

In [ ]:
# Not using
def toCLAHEgreen(img):  
    clipLimit=2.0 
    tileGridSize=(8, 8)  
    img = np.array(img)     
    green_channel = img[:, :, 1]    
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    cla = clahe.apply(green_channel) 
    cla=clahe.apply(cla)
    cla =cv2.merge((cla, cla, cla))
    return cla


## Applying cropping and Gaussian blur of SigmaX = 10 (Ben's Preprocessing)

---




In [ ]:
def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    # image = toCLAHEgreen(image)
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    # image = toCLAHEgreen(image)
        
    return image

## Create and output datasets

Train dataset






In [ ]:
save_dir = "/content/train_images_ben_preprocessed_sigmaX10"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for idx, row in tqdm_notebook(df_train.iterrows()):
    path=f"/content/train_images/{row['id_code']}.png"
    image = load_ben_color(path,sigmaX=10)
    Image.fromarray(image).save(os.path.join(save_dir, "{}.png".format(row['id_code'])))

Test dataset

In [ ]:
save_dir = "/content/test_images_ben_preprocessed_sigmaX10"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for idx, row in tqdm_notebook(df_test.iterrows()):
    path=f"/content/test_images/{row['id_code']}.png"
    image = load_ben_color(path,sigmaX=10)
    Image.fromarray(image).save(os.path.join(save_dir, "{}.png".format(row['id_code'])))
